# Web browser automation

Objective: Automate a certain mouse catching web game using Selenium on a local computer.

Username: mousehunt account username (string)

Password: mousehunt account password (string)

token: telegram bot token (string)

path: not in use currently.

In [1]:
# import all the stuff!

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from random import randint
import time
import requests
import json

# set login username and password

In [40]:
username = ''
password = ''
token = ''
last_updated_id = 0
path = "" 


In [2]:
def login(username, password):
    '''opens a chrome browser, and logs into selenium with a username and password. 
    assumes that the user currently has an account.
    returns driver object on successful login.'''
    try:   
        driver = webdriver.Chrome()
        driver.get("http://www.mousehuntgame.com")
        sign_in_text = driver.find_element_by_class_name('signInText')
        sign_in_text.click()
        #wait for scrollingContainer
        driver.implicitly_wait(2)
        login = driver.find_elements_by_class_name('scrollingContainer')[1] #select the form for existing user log in.
        # Why is it [1]? Only way I could make it work. find_elements unable to take arguments with a space in them for some reason.
        login_username = login.find_element_by_name('username')
        login_username.send_keys(username)
        login_password = login.find_element_by_name('password')
        login_password.send_keys(password)
        login.find_element_by_class_name('actionButton').click()
    except:
        return False
    return driver
    

In [4]:
def sound_the_horn(driver):
    # Click the hunters horn in HUD and return True. returns false if no hunters horn found.
    try:
        driver.find_element_by_class_name('mousehuntHud-huntersHorn').click()
    except:
        return False
    return True


# Need to find out: What are the elements present during kings reward?


In [5]:
def check_kings_reward(driver):
    # check for kings reward. Currently not working as intended, it always returns True.
    try:
        driver.find_element_by_class_name('mousehuntPage-puzzle-formContainer')
    except:
        return False
    return True


In [6]:
# find next hunt time
def next_hunt_time(driver, delay_minimum=0, delay_maximum=60):
    # returns the number of seconds till the next horn.
    # can add a random number of additional seconds to the next horn. default set as between 0,60 for safety.
    
    seconds = 0
    current_time = driver.find_element_by_class_name('mousehuntHud-hornTimer').text #grab current time in HUD
    # current time is a string, convert it into integer. (seconds)
    if current_time == "Ready!":
        return 0
    if current_time[1] == ":":
        #current time is less than 10:00
        minutes_to_second = 60*int(current_time[0:1])
        seconds += minutes_to_second
        seconds += int(current_time[-2:])
    else: # current time => 10:00
        minutes_to_second = 60*int(current_time[0:2]) 
        seconds += minutes_to_second
        seconds += int(current_time[-2:]) 
    
    # add some random amount of seconds to simulate randomness. by default, no seconds added.
    seconds += randint(delay_minimum, delay_maximum)
    print("Seconds till next hunt is: ", seconds)
    return seconds

    
    
        
        


In [7]:
#last_updated_id = 123123
def ask_user_clear_kr(last_updated_id, token):
    '''This function returns the solved kings reward, and the message ID used to solve that reward. 
    last_updated_id: function remembers the last input received by user, and checks for new inputs
    token: token for telegram bot.'''
    url = 'https://api.telegram.org/bot%s/' % token

    get_updates = json.loads(requests.get(url + 'getUpdates').content)
    print (get_updates['result'][-1]['message']['from']['id'])


    send_to_user_id = (get_updates['result'][-1]['message']['from']['id'])
    # Ping user once
    requests.get(url + 'sendMessage', params=dict(chat_id=send_to_user_id, text="Horn has failed"))


    
    # We want to keep checking for updates. So this must be a never ending loop
    while True:
    # My chat is up and running, I need to maintain it! Get me all chat updates
        get_updates = json.loads(requests.get(url + 'getUpdates').content)
        # check the id of last received message, compare to ID of latest message in stack.
        if last_updated_id == (get_updates['result'][-1]['update_id']):
            # if true, implies user has not responded. wait 60 seconds, check again.
            print("Same ID.")
            
            sleep(60)
        else:
            # if false, we have a new message. grab this message, store in variable.
            solved_kr_string = get_updates['result'][-1]['message']['text']
            print (solved_kr_string)
            last_updated_id = get_updates['result'][-1]['update_id']
            return solved_kr_string, last_updated_id
            

Run the below cell to start the process.

In [16]:
driver = login(username,password)

time.sleep(2)

while True:
    time.sleep(next_hunt_time(driver)) #sleeps for an amount of time between 0 to 15 minutes 
    sound_the_horn(driver)
    if sound_the_horn(driver == False):
        # when exception occur, means kings reward probably. 
        driver.refresh()
        #take a screenshot of kings reward.
        driver.save_screenshot(path)
        # send screenshot via telegram.
        solved_kr_string, last_updated_id = ask_user_clear_kc(last_updated_id, token)
        
        # 
        # return to start of loop.

NameError: name 'username' is not defined